<a href="https://colab.research.google.com/github/samekran/OptionsAlgo/blob/main/CallPuts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Global Vars
# Function to get user input for global variables
def get_global_vars():
    today_date_str = input("Enter today's date: ")
    expr_date_str = input("Enter the expiration date: ")

    time_frame_type = input("Enter the time frame type (Enter $ for dollar return or % for percentage change): ")
    time_frame_str = input("Enter the time frame (1 mo, YTD, TTM, 2023, 2022, 2021, All): ")

    time_frame_dict = {
        "$": {
            '1 mo': 10,
            'YTD': 11,
            'TTM': 12,
            '2023': 13,
            '2022': 14,
            '2021': 15,
            'All': 16
        },
        "%": {
            '1 mo': 19,
            'YTD': 20,
            'TTM': 21,
            '2023': 22,
            '2022': 23,
            '2021': 24,
            'All': 25
        }
    }

    time_frame = time_frame_dict[time_frame_type][time_frame_str]

    return today_date_str, expr_date_str, time_frame

# Get user input for global variables
today_date_str, expr_date_str, time_frame = get_global_vars()

print(f"Today's date: {today_date_str}")
print(f"Expiration date: {expr_date_str}")
print(f"Time frame: {time_frame}")


Enter today's date: 05-24-2024
Enter the expiration date: 05-31-2024
Enter the time frame type (Enter $ for dollar return or % for percentage change): %
Enter the time frame (1 mo, YTD, TTM, 2023, 2022, 2021, All): TTM
Today's date: 05-24-2024
Expiration date: 05-31-2024
Time frame: 21


In [ ]:
#@title Installs
# Import necessary libraries
!pip install scikit-optimize
!pip install openpyxl
!pip install matplotlib PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
#@title Imports
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import os
from sklearn.model_selection import train_test_split
from skopt.space import Real, Categorical
from skopt.utils import use_named_args
from sklearn.model_selection import KFold
from skopt import gp_minimize
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill, Font
from openpyxl.utils.dataframe import dataframe_to_rows
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import PyPDF2
from collections import deque
from joblib import Parallel, delayed
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from itertools import product
import csv

In [ ]:
#@title Upload Files
# Function to handle uploaded files
def upload_file():
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]  # Get the filename of the uploaded file
    return file_path

# Upload the main data file
print("Please upload the main data CSV file.")
main_data_file_path = upload_file()

# Upload the zipped folder containing the option data files
print("Please upload the zipped folder containing the option data files.")
zipped_folder_path = upload_file()

# Specify a directory name for the extracted files
extracted_folder = '/content/option_data'

# Unzip the folder
with zipfile.ZipFile(zipped_folder_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print("Files uploaded and unzipped successfully.")

Please upload the main data CSV file.


Saving Positive-5_pctchg_hibeta_sma20_4_0.25_30_N0_01_lower_2_2_NONE_0_below_timeline.csv to Positive-5_pctchg_hibeta_sma20_4_0.25_30_N0_01_lower_2_2_NONE_0_below_timeline.csv
Please upload the zipped folder containing the option data files.


Saving updated_may_24_2024_optionsdata.zip to updated_may_24_2024_optionsdata.zip
Files uploaded and unzipped successfully.


In [ ]:
#@title Setup
# Read the main data file
df = pd.read_csv(main_data_file_path)

# Get the list of stocks in the main CSV
csv_stocks = [df.columns[i + 2].split('_')[0] for i in range(1, len(df.columns) - 4, 3)]
print(f"Stocks in main CSV: {csv_stocks}")

# Get the list of stocks in the zip files
zip_stocks = set()
for filename in os.listdir(extracted_folder):
    stock_name = filename.split('_')[0]
    zip_stocks.add(stock_name)
zip_stocks = list(zip_stocks)
print(f"Stocks in zip files: {zip_stocks}")

# Find the intersection of stocks
all_stocks = list(set(csv_stocks) & set(zip_stocks))
all_stocks = sorted(all_stocks)
print(f"Combined list of stocks: {all_stocks}")

# Function to remove rows with volume less than 10% of the volume closest to the current price
def filter_option_data_by_volume(extracted_folder, main_data_file_path, all_stocks):
    # Read the main data file
    main_df = pd.read_csv(main_data_file_path)

    for stock in all_stocks:
        # Get the current price for the stock
        current_price = main_df[f'{stock}_adjclose'].iloc[0]

        for filename in os.listdir(extracted_folder):
            if filename.startswith(f"{stock}_"):
                file_path = os.path.join(extracted_folder, filename)
                df = pd.read_csv(file_path)

                if 'strike' in df.columns and 'volume' in df.columns:

                    # Filter out rows with NaN volumes
                    df_filtered = df.dropna(subset=['volume'])

                    # If the DataFrame is empty after filtering, skip to the next file
                    if df_filtered.empty:
                        print(f"No rows with non-NaN volumes found for stock {stock}.")
                        continue

                    # Find the closest strike price to the current price
                    df_filtered = df_filtered.copy()  # Make a copy to avoid SettingWithCopyWarning
                    df_filtered['diff'] = abs(df_filtered['strike'] - current_price)
                    closest_strike_row = df_filtered.loc[df_filtered['diff'].idxmin()]
                    closest_strike_volume = closest_strike_row['volume']

                    # Calculate the volume threshold
                    volume_threshold = 0.1 * closest_strike_volume

                    # Filter the DataFrame
                    df_filtered = df_filtered[df_filtered['volume'] >= volume_threshold]

                    # Save the filtered DataFrame back to the file
                    df_filtered.to_csv(file_path, index=False)

    print("Option data filtered by volume successfully.")

# Filter the option data by volume
filter_option_data_by_volume(extracted_folder, main_data_file_path, all_stocks)

Stocks in main CSV: ['AFRM', 'COIN', 'CFLT', 'CVNA', 'RIVN', 'TOST', 'CZR', 'W', 'PATH', 'RBLX', 'CAVA', 'DASH', 'SQ', 'CCL', 'MTDR', 'TTD', 'ON', 'NVDA', 'LNW', 'MPWR', 'GFS', 'LRCX', 'MKSI', 'MGM', 'OVV', 'ABNB', 'AMAT', 'CROX', 'DUOL', 'MSTR', 'RKT', 'MRVL', 'LSCC', 'BLDR', 'APA', 'AMD', 'KLAC', 'MCHP', 'DFS', 'FND', 'PINS', 'ENTG', 'GPS', 'AMP', 'WCC', 'HUBS', 'TSLA', 'PAYC', 'ONTO', 'XPO', 'DDS', 'TPR', 'WEX']
Stocks in zip files: ['', 'PEP', 'AMGN', 'AVGO', 'DASH', 'MSTR', 'DFS', 'PATH', 'TSLA', 'AAPL', 'NVDA', 'MGM', 'CLF', 'CROX', 'MDB', 'XLC', 'LLY', 'VRTX', 'CCL', 'RKT', 'HOOD', 'TTD', 'APP', 'KLAC', 'IOT', 'PANW', 'MSFT', 'SPY', 'AFRM', 'COIN', 'W', 'C', 'NFLX', 'INTC', 'AMAT', 'FCX', 'SQ', 'SNOW', 'MRVL', 'META', 'RBLX', 'APA', 'ENPH', 'PLTR', 'XOM', 'U', 'CI', 'TPR', 'NVO', 'UNH', 'CZR', 'XLF', 'OXY', 'PINS', 'AMD', 'CVNA', 'ON', 'MS', 'GPS', 'GOOGL', 'XLY', 'RCL', 'ALGN', 'RIVN', 'XLK', 'JPM', 'GTLB', 'GS', 'ABNB', 'CAVA', 'CVX', 'TOST', 'MRNA', 'LRCX', 'KO']
Combined lis

In [ ]:
#@title Filter Out Bad Signal Quality Stocks
# Read the main data file
df = pd.read_csv(main_data_file_path)

# Get the number of standard deviations to consider
num_std_dev = int(input("Enter the number of standard deviations (1 or 2): "))

# Calculate the number of relevant columns (total columns - 4 garbage columns)
numCols = len(df.columns) - 4

# Initialize dictionaries to store the results
price_changes = {}
signal_changes = {}

# Calculate price changes
for i in range(3, numCols, 3):
    price_column = df.columns[i]
    truncated_key = price_column.split('_')[0]
    price_changes[truncated_key] = []

    for j in range(len(df) - 5):
        current_price = df.iloc[j, i]
        future_price = df.iloc[j-5, i]
        if pd.notna(current_price) and pd.notna(future_price) and current_price != 0:
            percent_change = (future_price - current_price) / current_price * 100
            price_changes[truncated_key].append(percent_change)

# Calculate signal changes
for i in range(1, numCols, 3):
    signal_column = df.columns[i]
    truncated_key = signal_column.split('_')[0]
    signal_changes[truncated_key] = []

    for j in range(len(df) - 5):
        signal_value = df.iloc[j, i]
        if signal_value == 1.0 and j > 4:
            price_change = df.iloc[j, i + 1]
            signal_changes[truncated_key].append(price_change)

# Function to calculate percentages for given sigma
def calculate_percentages(data, mean, std, sigma):
    above_sigma = np.sum(np.array(data) > (mean + sigma * std)) / len(data) * 100
    return above_sigma

# Filter stocks based on signal quality
stocks_to_keep = []

threshold = float(input("Enter how much higher the signal tail should be (ex: 0.1 for 10%): "))

for key in all_stocks:
    if key in signal_changes and key in price_changes:
        price_data = price_changes[key]
        signal_data = signal_changes[key]

        mean_price = np.mean(price_data)
        std_price = np.std(price_data)

        price_above_sigma = calculate_percentages(price_data, mean_price, std_price, num_std_dev)
        signal_above_sigma = calculate_percentages(signal_data, mean_price, std_price, num_std_dev)

        if price_above_sigma != 0:
            ratio = signal_above_sigma / price_above_sigma
            if ratio >= 1+threshold:
                stocks_to_keep.append(key)

# Update the all_stocks list
all_stocks = sorted(stocks_to_keep)

print(f"Filtered list of stocks: {all_stocks}")

Enter the number of standard deviations (1 or 2): 2
Enter how much higher the signal tail should be (ex: 0.1 for 10%): 0.1
Filtered list of stocks: ['AAPL', 'ABNB', 'AFRM', 'AVGO', 'C', 'CCL', 'CI', 'CLF', 'COIN', 'CROX', 'CVX', 'DASH', 'DFS', 'GOOGL', 'GPS', 'GTLB', 'HOOD', 'INTC', 'KO', 'LLY', 'MGM', 'MRVL', 'MSTR', 'PANW', 'PATH', 'PEP', 'PINS', 'RBLX', 'RIVN', 'RKT', 'SPY', 'TOST', 'TPR', 'TSLA', 'TTD', 'UNH', 'XLC', 'XOM']


In [ ]:
#@title Change days
# Function to calculate percent change for the specified day range
def calculate_percent_change(df, days):
    # Ensure the day range is valid
    if days <= 0:
        raise ValueError("Day range must be greater than 0.")

    # Iterate over each stock (assumed to be every 3 columns)
    numCols = len(df.columns) - 4
    for i in range(1, numCols, 3):
        signal_col = df.columns[i]  # Signal column
        pct_change_col = df.columns[i + 1]  # Percent change column
        adj_close_col = df.columns[i + 2]  # Adjusted close column

        # Initialize the pct change column
        df[pct_change_col] = [np.nan] * len(df)

        for j in range(len(df)):
            if j >= days:
                old_price = df.at[j, adj_close_col]
                new_price = df.at[j - days, adj_close_col]
                if old_price != 0:  # Avoid division by zero
                    pct_change = (new_price - old_price) / old_price * 100
                    df.at[j, pct_change_col] = pct_change
                else:
                    df.at[j, pct_change_col] = 0.0
    return df

# Function to calculate the number of weekdays between two dates
def calculate_weekdays(start_date_str, end_date_str):
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)
    weekdays = pd.date_range(start=start_date, end=end_date, freq='B')
    return len(weekdays)-1

# Read the main data file
df = pd.read_csv(main_data_file_path)

# Calculate the number of weekdays between today_date_str and expr_date_str
days = calculate_weekdays(today_date_str, expr_date_str)

# Calculate percent change
df = calculate_percent_change(df, days)

print(days)

# Save the updated DataFrame to a new CSV file
main_data_file_path = 'updated_main_data.csv'
df.to_csv(main_data_file_path, index=False)

print(f"Updated CSV saved to {main_data_file_path}")


5
Updated CSV saved to updated_main_data.csv


In [ ]:
#@title get_df
def get_percents(ante_price, ticker, time_frame, dummy_df, all_days, call_buy=None, call_sell=None, put_sell=None, put_buy=None):
    # Define a function to parse dates
    def parse_dates(date):
        for fmt in ("%Y-%m-%d", "%m/%d/%y"):
            try:
                return pd.to_datetime(date, format=fmt)
            except ValueError:
                continue
        # If no format matches, raise an error
        raise ValueError(f"Date {date} is not in a recognized format")

    # Read the main data file
    df = pd.read_csv(main_data_file_path)

    # Parse the dates
    df['date'] = df['date'].apply(parse_dates)

    # Get the current date
    current_date = df['date'].iloc[0]

    # Filter data according to the time frame
    if time_frame == '1 Mo':
        start_date = current_date - pd.DateOffset(months=1)
    elif time_frame == 'YTD':
        start_date = pd.Timestamp(year=current_date.year, month=1, day=1)
    elif time_frame == 'TTM':
        start_date = current_date - pd.DateOffset(years=1)
    elif time_frame == '2023':
        start_date = pd.Timestamp(year=2023, month=1, day=1)
        end_date = pd.Timestamp(year=2023, month=12, day=31)
    elif time_frame == '2022':
        start_date = pd.Timestamp(year=2022, month=1, day=1)
        end_date = pd.Timestamp(year=2022, month=12, day=31)
    elif time_frame == '2021':
        start_date = pd.Timestamp(year=2021, month=1, day=1)
        end_date = pd.Timestamp(year=2021, month=12, day=31)
    elif time_frame == 'All':
        start_date = df['date'].min()
    else:
        raise ValueError("Invalid time frame specified")

    # Apply the date filter
    if time_frame in ['2023', '2022', '2021']:
        df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    else:
        df = df[df['date'] >= start_date]

    # Get the current price
    current_price = df[f'{ticker}_adjclose'].iloc[-1]

    # Initialize variables
    win = 0.0
    loss = 0.0
    max_loss = 0.0
    ante = 0.0

    win_dollar = 0.0
    loss_dollar = 0.0
    max_loss_dollar = 0.0
    ante_dollar = 0.0

    win_count = 0
    loss_count = 0
    max_loss_count = 0
    if dummy_df.iloc[5, 2] == 0:
        return win, loss, max_loss, ante, win_dollar, loss_dollar, max_loss_dollar, ante_dollar, win_count, loss_count, max_loss_count

    # Adjust for positive ante
    if dummy_df.iloc[5, 1] > 0:
        numerator = ante_price
    else:
        numerator = -ante_price

    for idx, row in df.iterrows():
        if all_days or row[f'{ticker}_signal_exists'] == 1:
            pct_chg = row[f'{ticker}_5_pctchg'] / 100
            conversion_rate = numerator / (float(dummy_df.iloc[5, 1]) * row[f'{ticker}_adjclose'])
            total = 0.0
            #Call buy
            if call_buy is not None:
                total += max(pct_chg-call_buy,0)
            #Call sell
            if call_sell is not None:
                total -= max(pct_chg-call_sell,0)
            #Put buy
            if put_buy is not None:
                total += max(put_buy-pct_chg,0)
            #Put sell
            if put_sell is not None:
                total -= max(put_sell-pct_chg,0)

            if total > 0:
                win += total
                win_dollar += total*row[f'{ticker}_adjclose'] * conversion_rate
                win_count += 1

            if total < 0:
                loss += total
                loss_dollar += total*row[f'{ticker}_adjclose'] * conversion_rate
                loss_count += 1

            ante += 1
            ante_dollar += float(dummy_df.iloc[5, 2]) * row[f'{ticker}_adjclose']

    return win, loss, max_loss, ante, win_dollar, loss_dollar, max_loss_dollar, ante_dollar, win_count, loss_count, max_loss_count

def find_closest_strike(ticker, target_price, option_type, current_price):
    closest_strike = None
    closest_diff = float('inf')
    average_bid_ask = None
    option_type_alternatives = [option_type, option_type.lower() + 's']  # E.g., 'CALL' and 'calls' or 'PUT' and 'puts'

    for filename in os.listdir(extracted_folder):
        if any(f"{ticker}_{today_date_str}_{alt}_exp_{expr_date_str}" in filename for alt in option_type_alternatives):
            file_path = os.path.join(extracted_folder, filename)
            df = pd.read_csv(file_path)
            if 'strike' in df.columns and 'bid' in df.columns and 'ask' in df.columns:
                df['diff'] = abs(df['strike'] - target_price)
                min_diff_row = df.loc[df['diff'].idxmin()]
                if min_diff_row['diff'] < closest_diff:
                    closest_diff = min_diff_row['diff']
                    closest_strike = min_diff_row['strike']
                    average_bid_ask = (min_diff_row['bid'] + min_diff_row['ask']) / 2

    return closest_strike, average_bid_ask


def get_df(ticker, call_buy=None, call_sell=None, put_sell=None, put_buy=None, all_days=False):
    # Read the main data file
    df = pd.read_csv(main_data_file_path)

    # Get the current price from the most recent price in the [ticker]_adjclose column
    current_price = df[f'{ticker}_adjclose'].iloc[0]

    # Step 1: Define the headers
    headers = [f'{ticker} Stock', 'Strike', '% change', '5 day option price', 'Column 5', 'Column 6']

    # Step 2: Define the first column data
    first_column_data = [
        ["Current price"],
        ["Call Buy"],
        ["Call Sell"],
        ["Put 1 (sell)"],
        ["Put 2 (buy)"],
        ["Net Ante"],
        [""],
        [""],
        ["Current Option Price"],
        ["$100 Ante"],
        ["1 Mo"],
        ["YTD"],
        ["TTM"],
        ["2023"],
        ["2022"],
        ["2021"],
        ["All"],
        [""],
        ["Percent Change"],
        ["1 Mo"],
        ["YTD"],
        ["TTM"],
        ["2023"],
        ["2022"],
        ["2021"],
        ["All"],
        [""],
        ["Win Rate"],
        ["1 Mo"],
        ["YTD"],
        ["TTM"],
        ["2023"],
        ["2022"],
        ["2021"],
        ["All"]
    ]

    # Convert to DataFrame
    first_column_df = pd.DataFrame(first_column_data, columns=[f'{ticker} Stock'])

    # Step 3: Create dummy data for the remaining columns
    num_rows = len(first_column_data)
    dummy_data = {
        'Strike': [''] * num_rows,
        '% change': [''] * num_rows,
        '5 day option price': [''] * num_rows,
        'Column 5': [''] * num_rows,
        'Column 6': [''] * num_rows
    }

    # Find the closest strikes and average bid/ask for the given targets
    call_buy_target_price = current_price * (1 + call_buy) if call_buy is not None else None
    closest_call_buy_strike, average_call_buy_bid_ask = find_closest_strike(ticker, call_buy_target_price, 'CALL', current_price) if call_buy is not None else (None, None)

    call_sell_target_price = current_price * (1 + call_sell) if call_sell is not None else None
    closest_call_sell_strike, average_call_sell_bid_ask = find_closest_strike(ticker, call_sell_target_price, 'CALL', current_price) if call_sell is not None else (None, None)

    putsell_target_price = current_price * (1 + put_sell) if put_sell is not None else None
    closest_putsell_strike, average_putsell_bid_ask = find_closest_strike(ticker, putsell_target_price, 'PUT', current_price) if put_sell is not None else (None, None)

    putbuy_target_price = current_price * (1 + put_buy) if put_buy is not None else None
    closest_putbuy_strike, average_putbuy_bid_ask = find_closest_strike(ticker, putbuy_target_price,'PUT', current_price) if put_buy is not None else (None, None)

    # Convert dummy data to DataFrame
    dummy_df = pd.DataFrame(dummy_data)
    dummy_df.at[0, 'Strike'] = current_price

    # Assign values to specific cells
    if call_buy is not None:
        dummy_df.at[1, 'Strike'] = closest_call_buy_strike
        dummy_df.at[1, '5 day option price'] = average_call_buy_bid_ask
        dummy_df.at[1, '% change'] = closest_call_buy_strike / current_price - 1

    if call_sell is not None:
        dummy_df.at[2, 'Strike'] = closest_call_sell_strike
        dummy_df.at[2, '5 day option price'] = average_call_sell_bid_ask
        dummy_df.at[2, '% change'] = closest_call_sell_strike / current_price - 1

    if put_sell is not None:
        dummy_df.at[3, 'Strike'] = closest_putsell_strike
        dummy_df.at[3, '5 day option price'] = average_putsell_bid_ask
        dummy_df.at[3, '% change'] = closest_putsell_strike / current_price - 1

    if put_buy is not None:
        dummy_df.at[4, 'Strike'] = closest_putbuy_strike
        dummy_df.at[4, '5 day option price'] = average_putbuy_bid_ask
        dummy_df.at[4, '% change'] = closest_putbuy_strike / current_price - 1

    # Net ante
    dummy_df.at[5, '5 day option price'] = (
        (float(dummy_df.at[3, '5 day option price']) if put_sell is not None and dummy_df.at[3, '5 day option price'] else 0)
        - (float(dummy_df.at[1, '5 day option price']) if call_buy is not None and dummy_df.at[1, '5 day option price'] else 0)
        - (float(dummy_df.at[4, '5 day option price']) if put_buy is not None and dummy_df.at[4, '5 day option price'] else 0)
        + (float(dummy_df.at[2, '5 day option price']) if call_sell is not None and dummy_df.at[2, '5 day option price'] else 0)
    )
    dummy_df.at[5, '% change'] = float(dummy_df.at[5, '5 day option price']) / float(dummy_df.at[0, 'Strike']) if dummy_df.at[0, 'Strike'] else 0.0

    # Define the time frames to process
    time_frames = ['1 Mo', 'YTD', 'TTM', '2023', '2022', '2021', 'All']
    row_mapping = {
        '1 Mo': 19,
        'YTD': 20,
        'TTM': 21,
        '2023': 22,
        '2022': 23,
        '2021': 24,
        'All': 25
    }

    row_mapping_dollar = {
        '1 Mo': 10,
        'YTD': 11,
        'TTM': 12,
        '2023': 13,
        '2022': 14,
        '2021': 15,
        'All': 16
    }

    row_mapping_rate = {
        '1 Mo': 28,
        'YTD': 29,
        'TTM': 30,
        '2023': 31,
        '2022': 32,
        '2021': 33,
        'All': 34
    }

    ante_price = 100
    flag = True
    while flag:
        flag = False
        for time_frame in time_frames:
            win, loss, max_loss, ante, win_dollar, loss_dollar, max_loss_dollar, ante_dollar, win_count, loss_count, max_loss_count = get_percents(ante_price, ticker, time_frame, dummy_df, all_days, call_buy, call_sell, put_sell, put_buy)
            if np.abs(max_loss_dollar) > 3 * np.abs(ante_dollar):
                flag = True
                break
            row = row_mapping[time_frame]
            dummy_df.at[row, 'Strike'] = win
            dummy_df.at[row, '% change'] = loss
            dummy_df.at[row, '5 day option price'] = max_loss
            dummy_df.at[row, 'Column 5'] = ante

            row_dollar = row_mapping_dollar[time_frame]
            dummy_df.at[row_dollar, 'Strike'] = win_dollar
            dummy_df.at[row_dollar, '% change'] = loss_dollar
            dummy_df.at[row_dollar, '5 day option price'] = max_loss_dollar
            dummy_df.at[row_dollar, 'Column 5'] = ante_dollar

            row_rate = row_mapping_rate[time_frame]
            total = win_count + loss_count + max_loss_count
            if total != 0:
                dummy_df.at[row_rate, 'Strike'] = win_count / total
                dummy_df.at[row_rate, '% change'] = loss_count / total
                dummy_df.at[row_rate, '5 day option price'] = max_loss_count / total

        if flag:
            ante_price = ante_dollar / (3 * max_loss_dollar)

    # Get percent totals
    for i in range(19, 26):
        total = 0
        for j in range(0, 4):
            try:
                if j == 3:
                    total += float(dummy_df.iloc[i, j]) * dummy_df.iloc[5, 1]
                else:
                    total += float(dummy_df.iloc[i, j])
            except ValueError:
                continue  # Skip non-numeric values
        dummy_df.at[i, 'Column 6'] = total

    # $100 Ante
    for i in range(10, 17):
        if dummy_df.iloc[5, 1] > 0:
            dummy_df.at[i, 'Column 5'] = 100 * dummy_df.at[i + 9, 'Column 5']
        else:
            dummy_df.at[i, 'Column 5'] = -100 * dummy_df.at[i + 9, 'Column 5']

    # Get price totals
    for i in range(10, 17):
        total = 0
        for j in range(0, 4):
            try:
                total += float(dummy_df.iloc[i, j])
            except ValueError:
                continue  # Skip non-numeric values
        dummy_df.at[i, 'Column 6'] = total

    dummy_df.iloc[9, 0] = 'Win $'
    dummy_df.iloc[9, 1] = 'Loss $'
    dummy_df.iloc[9, 2] = 'Max Loss $'
    dummy_df.iloc[9, 3] = 'Ante $'
    dummy_df.iloc[9, 4] = 'Net $'

    dummy_df.iloc[18, 0] = 'Win %'
    dummy_df.iloc[18, 1] = 'Loss %'
    dummy_df.iloc[18, 2] = 'Max Loss %'
    dummy_df.iloc[18, 3] = 'Ante #'
    dummy_df.iloc[18, 4] = 'Net %'

    dummy_df.iloc[27, 0] = 'Win %'
    dummy_df.iloc[27, 1] = 'Loss %'
    dummy_df.iloc[27, 2] = 'Max Loss %'

    # Step 4: Combine the first column DataFrame with the dummy data DataFrame
    combined_df = pd.concat([first_column_df, dummy_df], axis=1)

    # Add headers to the combined DataFrame
    combined_df.columns = headers

    return combined_df

In [ ]:
#@title portfolio optimize
def portfolio_optimize(all_days):

    # Define the parameter space
    space = [
        Categorical([None] + list(np.linspace(0.005, 0.2, num=20)), name='call_buy'),
        Categorical([None] + list(np.linspace(-0.2, -0.01, num=20)), name='call_sell'),
        Categorical([None] + list(np.linspace(-0.1, -0.01, num=10)), name='put_sell'),
        Categorical([None] + list(np.linspace(-0.2, -0.01, num=20)), name='put_buy')
    ]

    def run_analysis(stocks, call_buy, call_sell, put_sell, put_buy, time_frame):
        gain_values_pos_ante = []
        gain_values_neg_ante = []

        # Define the function to process each stock
        def process_stock(stock):
            stock_df = get_df(stock, call_buy, call_sell, put_sell, put_buy, all_days)
            if stock_df is not None:
                try:
                    ante_value = stock_df.iloc[5, 2]
                    gain_value = stock_df.iloc[time_frame, 5]
                    # Check if the ante value is negative
                    if ante_value < 0:
                        return None, gain_value
                    else:
                        return gain_value, None
                except (IndexError, KeyError):
                    return None, None
            return None, None

        # Use parallel processing to process stocks
        results = Parallel(n_jobs=-1)(delayed(process_stock)(stock) for stock in stocks)
        #results = [process_stock(stock) for stock in stocks]

        # Separate positive and negative ante results
        for gain_pos, gain_neg in results:
            if gain_pos is not None:
                gain_values_pos_ante.append(gain_pos)
            if gain_neg is not None:
                gain_values_neg_ante.append(gain_neg)

        # Calculate the average gain value for positive ante
        if gain_values_pos_ante:
            average_gain_pos_ante = sum(gain_values_pos_ante) / len(gain_values_pos_ante)
        else:
            average_gain_pos_ante = None

        # Calculate the average gain value for negative ante
        if gain_values_neg_ante:
            average_gain_neg_ante = sum(gain_values_neg_ante) / len(gain_values_neg_ante)
        else:
            average_gain_neg_ante = None

        return average_gain_pos_ante, average_gain_neg_ante

    # Split the stocks into training and validation sets
    train_stocks, val_stocks = train_test_split(all_stocks, test_size=0.2, random_state=42)

    # Define the objective function for positive ante
    @use_named_args(space)
    def objective_pos_ante(**params):
        call_buy = params.get('call_buy', None)
        call_sell = params.get('call_sell', None)
        put_sell = params.get('put_sell', None)
        put_buy = params.get('put_buy', None)
        # Ensure valid combinations of options
        if call_buy is None or (call_sell is not None and call_sell >= call_buy) or put_buy is None:
            return 0.0

        gain_pos_ante, _ = run_analysis(train_stocks, call_buy, call_sell, put_sell, put_buy, time_frame)
        if not isinstance(gain_pos_ante, (int, float)) or np.isnan(gain_pos_ante):
            return 0.0
        return -gain_pos_ante  # Minimize negative gain to maximize gain

    # Define the objective function for negative ante
    @use_named_args(space)
    def objective_neg_ante(**params):
        call_buy = params.get('call_buy', None)
        call_sell = params.get('call_sell', None)
        put_sell = params.get('put_sell', None)
        put_buy = params.get('put_buy', None)
        # Ensure valid combinations of options
        if call_buy is None or (call_sell is not None and call_sell >= call_buy) or put_buy is None:
            return 0.0

        _, gain_neg_ante = run_analysis(train_stocks, call_buy, call_sell, put_sell, put_buy, time_frame)
        if not isinstance(gain_neg_ante, (int, float)) or np.isnan(gain_neg_ante):
            return 0.0
        return -gain_neg_ante  # Minimize negative gain to maximize gain

    # Perform Bayesian optimization for positive ante
    result_pos_ante = gp_minimize(objective_pos_ante, space, n_calls=30, random_state=0)
    print("Best parameters found for positive ante: call_buy={}, call_sell={}, put_sell={}, put_buy={}".format(result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3]))
    train_gain_pos_ante, _ = run_analysis(train_stocks, result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3], time_frame)
    print(f"Average gain after ante for training set with positive ante: {train_gain_pos_ante}")

    # Perform Bayesian optimization for negative ante
    result_neg_ante = gp_minimize(objective_neg_ante, space, n_calls=30, random_state=0)
    print("Best parameters found for negative ante: call_buy={}, call_sell={}, put_sell={}, put_buy={}".format(result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3]))
    _, train_gain_neg_ante = run_analysis(train_stocks, result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3], time_frame)
    print(f"Average gain after ante for training set with negative ante: {train_gain_neg_ante}")

    # Validate on the remaining 20% stocks
    val_gain_pos_ante = run_analysis(val_stocks, result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3], time_frame)
    val_gain_neg_ante = run_analysis(val_stocks, result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3], time_frame)
    print(f"Validation gain after ante for validation set with positive ante: {val_gain_pos_ante[0]}")
    print(f"Validation gain after ante for validation set with negative ante: {val_gain_neg_ante[1]}")

    return train_gain_pos_ante, train_gain_neg_ante, val_gain_pos_ante, val_gain_neg_ante, result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3], result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3]

In [ ]:
#@title run portfolio optimize
train_gain_pos_ante, train_gain_neg_ante, val_gain_pos_ante, val_gain_neg_ante, poscallbuy, poscallsell, posputsell, posputbuy, negcallbuy, negcallsell, negputsell, negputbuy = portfolio_optimize(False)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [ ]:
#@title individual optimize
def individual_optimize(all_days):
    def split_stock_df(df, stock_name):
        signal_indices = df.index[df[f'{stock_name}_signal_exists'] == 1].tolist()
        num_signals = len(signal_indices)
        if num_signals < 10:
            return None, None

        split_index = int(0.8 * num_signals)
        train_indices = signal_indices[:split_index]
        val_indices = signal_indices[split_index:]

        train_df = df.loc[train_indices].reset_index(drop=True)
        val_df = df.loc[val_indices].reset_index(drop=True)

        return train_df, val_df

    def run_analysis(stock_name, call_buy, call_sell, put_sell, put_buy, time_frame, all_days):
        stock_df = get_df(stock_name, call_buy, call_sell, put_sell, put_buy, all_days=all_days)
        if stock_df is not None:
            gain_value = stock_df.iloc[time_frame, 5]
            return gain_value
        return np.nan

    def optimize_for_stock(stock_name, train_df, val_df, combination, all_days):
        # Define the parameter space for the specific combination
        space = []
        if 'call_buy' in combination:
            space.append(Categorical(list(np.linspace(0.005, 0.2, num=20)), name='call_buy'))
        if 'call_sell' in combination:
            space.append(Categorical(list(np.linspace(-0.2, -0.01, num=20)), name='call_sell'))
        if 'put_sell' in combination:
            space.append(Categorical(list(np.linspace(-0.1, -0.01, num=10)), name='put_sell'))
        if 'put_buy' in combination:
            space.append(Categorical(list(np.linspace(-0.2, -0.01, num=20)), name='put_buy'))

        @use_named_args(space)
        def objective(**params):
            call_buy = params.get('call_buy', None)
            call_sell = params.get('call_sell', None)
            put_sell = params.get('put_sell', None)
            put_buy = params.get('put_buy', None)

            # Enforce constraints
            if call_buy is None or (call_sell is not None and call_sell >= call_buy) or put_buy is None:
                return 0.0

            gain = run_analysis(stock_name, call_buy, call_sell, put_sell, put_buy, time_frame, all_days)
            if np.isnan(gain):
                return 0.0
            return -gain

        result = gp_minimize(objective, space, n_calls=10, random_state=42)

        # Map the result back to the appropriate parameters
        call_buy_opt = None
        call_sell_opt = None
        put_sell_opt = None
        put_buy_opt = None

        result_index = 0
        if 'call_buy' in combination:
            call_buy_opt = result.x[result_index]
            result_index += 1
        if 'call_sell' in combination:
            call_sell_opt = result.x[result_index]
            result_index += 1
        if 'put_sell' in combination:
            put_sell_opt = result.x[result_index]
            result_index += 1
        if 'put_buy' in combination:
            put_buy_opt = result.x[result_index]

        val_gain = run_analysis(stock_name, call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt, time_frame, all_days)
        if np.isnan(val_gain):
            val_gain = 0.0  # Ensure no NaN values
        return call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt, val_gain

    def process_stock(stock, all_days):
        train_df, val_df = split_stock_df(df, stock)
        if train_df is not None and val_df is not None:
            best_combination = None
            best_gain = -float('inf')
            best_params = None

            # Define the valid combinations
            valid_combinations = [
                ['call_buy', 'put_buy'],
                ['call_buy', 'call_sell', 'put_buy'],
                ['call_buy', 'put_sell', 'put_buy'],
                ['call_buy', 'call_sell', 'put_sell', 'put_buy']
            ]

            for combination in valid_combinations:
                params = optimize_for_stock(stock, train_df, val_df, combination, all_days)
                if params[4] > best_gain:
                    best_gain = params[4]
                    best_params = params
                    best_combination = combination

            return best_params
        return None, None, None, None, None

    optimized_params = {}
    validation_gains = []

    df = pd.read_csv(main_data_file_path)  # Load the main data file

    # Use parallel computing to optimize for each stock
    results = Parallel(n_jobs=-1)(delayed(process_stock)(stock, all_days) for stock in all_stocks)

    for stock, result in zip(all_stocks, results):
        if result is not None:
            call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt, val_gain = result
            if not np.isnan(val_gain):
                optimized_params[stock] = (call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt)
                validation_gains.append(val_gain)
            else:
                print(f"Stock {stock} returned a NaN validation gain")
        else:
            print(f"Stock {stock} optimization returned None")

    if validation_gains:
        average_validation_gain = sum(validation_gains) / len(validation_gains)
    else:
        average_validation_gain = np.nan

    print(f"Average gain after ante for validation sets across all stocks: {average_validation_gain}")

    return optimized_params, validation_gains, average_validation_gain

In [ ]:
#@title run individual optimize
optimized_params, validation_gains, average_validation_gain = individual_optimize(all_days=False)

In [ ]:
#@title debugging
# Find stocks that are in optimized_params.keys() but not in all_stocks
missing_stocks = set(optimized_params.keys()) - set(all_stocks)

# Print the missing stocks
if len(missing_stocks) == 0:
  print("All stocks accounted for")
else:
  print("Missing stocks:")
  for stock in missing_stocks:
      print(stock)

In [ ]:
#@title manual optimize
def manual_optimize(all_days):
    # Define the parameter space with constraints
    def get_constrained_strikes(extracted_folder, ticker, option_type, current_price, max_steps=3):
        strikes = set()
        for filename in os.listdir(extracted_folder):
            parts = filename.split('_')
            if len(parts) >= 4 and parts[0] == ticker and parts[2] == option_type:
                file_path = os.path.join(extracted_folder, filename)
                df = pd.read_csv(file_path)
                if 'strike' in df.columns:
                    strikes.update(df['strike'].tolist())

        sorted_strikes = sorted(list(strikes))
        constrained_strikes = []

        for strike in sorted_strikes:
            if option_type == 'CALL' and strike > current_price:
                constrained_strikes.append(strike)
            elif option_type == 'PUT' and strike < current_price:
                constrained_strikes.append(strike)

            # Check if the number of steps is within the limit
            if len(constrained_strikes) > max_steps:
                break

        return [None] + constrained_strikes

    # Get available strikes for each ticker with constraints
    def get_all_constrained_strikes(extracted_folder, all_stocks, main_data_file_path):
        main_df = pd.read_csv(main_data_file_path)
        available_strikes = {}
        for stock in all_stocks:
            current_price = main_df[f'{stock}_adjclose'].iloc[0]
            available_strikes[stock] = {
                'CALL': get_constrained_strikes(extracted_folder, stock, 'CALL', current_price),
                'PUT': get_constrained_strikes(extracted_folder, stock, 'PUT', current_price)
            }
        return available_strikes

    available_strikes = get_all_constrained_strikes(extracted_folder, all_stocks, main_data_file_path)

    def run_analysis(stock_name, call_buy, put_sell, put_buy, time_frame, all_days):
        stock_df = get_df(stock_name, call_buy, None, put_sell, put_buy, all_days=all_days)
        if stock_df is not None:
            gain_value = stock_df.iloc[time_frame, 5]
            return gain_value
        return np.nan

    def optimize_for_stock(stock_name, current_price, train_df, val_df, all_days):
        # Get the available strikes for the stock with constraints
        call_strikes = available_strikes[stock_name]['CALL']
        put_strikes = available_strikes[stock_name]['PUT']

        best_params = None
        best_gain = -float('inf')

        # Try all combinations of strikes including None
        for call_buy, put_sell, put_buy in product(call_strikes, put_strikes, put_strikes):
            if call_buy is not None:
                call_buy_pct = (call_buy / current_price) - 1
            else:
                call_buy_pct = None

            if put_sell is not None:
                put_sell_pct = (put_sell / current_price) - 1
            else:
                put_sell_pct = None

            if put_buy is not None:
                put_buy_pct = (put_buy / current_price) - 1
            else:
                put_buy_pct = None

            # Ensure put buy is lower than put sell and constraints are met
            if call_buy is not None and put_sell is not None and put_buy is not None:
                if (call_buy > current_price) and (put_sell < current_price) and (put_buy < current_price) and (put_buy < put_sell):
                    gain = run_analysis(stock_name, call_buy_pct, put_sell_pct, put_buy_pct, time_frame, all_days)
                    if not np.isnan(gain) and gain > best_gain:
                        best_gain = gain
                        best_params = (call_buy_pct, put_sell_pct, put_buy_pct)

        if best_params is not None:
            return *best_params, best_gain
        else:
            return None, None, None, best_gain



    def split_stock_df(df, stock_name):
        signal_indices = df.index[df[f'{stock_name}_signal_exists'] == 1].tolist()
        num_signals = len(signal_indices)
        if num_signals < 10:
            return None, None

        split_index = int(0.8 * num_signals)
        train_indices = signal_indices[:split_index]
        val_indices = signal_indices[split_index:]

        train_df = df.loc[train_indices].reset_index(drop=True)
        val_df = df.loc[val_indices].reset_index(drop=True)

        return train_df, val_df

    # Perform the optimization for each stock
    manual_params = {}
    manual_validation_gains = []

    df = pd.read_csv(main_data_file_path)  # Load the main data file

    for stock in all_stocks:
        current_price = df[f'{stock}_adjclose'].iloc[0]
        train_df, val_df = split_stock_df(df, stock)
        if train_df is not None and val_df is not None:
            result = optimize_for_stock(stock, current_price, train_df, val_df, all_days)
            if result is not None:
                call_buy_opt, put_sell_opt, put_buy_opt, val_gain = result
                if not np.isnan(val_gain):
                    manual_params[stock] = (call_buy_opt, put_sell_opt, put_buy_opt)
                    manual_validation_gains.append(val_gain)
                else:
                    print(f"Stock {stock} returned a NaN validation gain")
            else:
                print(f"Stock {stock} optimization returned None")
        else:
            print(f"Stock {stock} has insufficient data for optimization")

    if manual_validation_gains:
        average_manual_gain = sum(manual_validation_gains) / len(manual_validation_gains)
    else:
        average_manual_gain = np.nan

    print(f"Average %gain after ante for validation sets across all stocks: {average_manual_gain}")

    return manual_params, manual_validation_gains, average_manual_gain


In [ ]:
#@title Run manual optimization
manual_params, manual_validation_gains, average_manual_gain = manual_optimize(all_days=False)

In [ ]:
#@title save parameters
output_csv = 'optimization_results.csv'


# Prepare data for writing
data = [
    ["Portfolio Optimize"],
    ["poscallbuy", poscallbuy],
    ["poscallsell", poscallsell],
    ["posputsell", posputsell],
    ["posputbuy", posputbuy],
    ["negcallbuy", negcallbuy],
    ["negcallsell", negcallsell],
    ["negputsell", negputsell],
    ["negputbuy", negputbuy],
    ["train_gain_pos_ante", train_gain_pos_ante],
    ["train_gain_neg_ante", train_gain_neg_ante],
    ["val_gain_pos_ante", val_gain_pos_ante],
    ["val_gain_neg_ante", val_gain_neg_ante],
    [],
    ["Individual Optimize"],
    ["optimized_params", str(optimized_params)],
    ["validation_gains", str(validation_gains)],
    ["average_validation_gain", average_validation_gain],
    [],
    ["Manual Optimize"],
    ["manual_params", str(manual_params)],
    ["manual_validation_gains", str(manual_validation_gains)],
    ["average_manual_gain", average_manual_gain]
]

# Write data to CSV
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"Results saved to {output_csv}")

NameError: name 'train_gain_pos_ante' is not defined

In [ ]:
poscallbuy = 0.13842
poscallsell = -0.01
posputsell = -0.1
posputbuy = -0.01
negcallbuy = 0.1282
negcallsell = -0.13
negputsell = -0.08
negputbuy = -0.2
optimized_params = {'ABNB': (0.005, None, -0.01, -0.16), 'AFRM': (0.005, None, -0.049999999999999996, -0.13), 'AMAT': (0.2, -0.11000000000000001, None, -0.03), 'AMD': (0.11789473684210527, -0.12000000000000001, None, -0.19), 'APA': (0.005, None, -0.049999999999999996, -0.13), 'CAVA': (0.005, -0.020000000000000018, -0.049999999999999996, -0.13), 'CCL': (0.13842105263157894, -0.19, None, -0.06), 'COIN': (0.11789473684210527, -0.12000000000000001, None, -0.19), 'CROX': (0.13842105263157894, -0.19, None, -0.06), 'CVNA': (0.005, None, -0.01, -0.16), 'CZR': (0.005, None, -0.01, -0.16), 'DASH': (0.005, None, -0.01, -0.16), 'DFS': (0.005, None, -0.01, -0.16), 'GPS': (0.005, None, -0.01, -0.16), 'KLAC': (0.2, -0.11000000000000001, None, -0.03), 'LRCX': (0.18973684210526315, -0.2, None, -0.01), 'MGM': (0.005, None, -0.01, -0.16), 'MRVL': (0.2, -0.11000000000000001, None, -0.03), 'MSTR': (0.18973684210526315, -0.2, None, -0.01), 'NVDA': (0.18973684210526315, -0.2, None, -0.01), 'ON': (0.11789473684210527, -0.12000000000000001, None, -0.19), 'PATH': (0.1281578947368421, None, -0.039999999999999994, -0.2), 'PINS': (0.005, None, -0.01, -0.16), 'RBLX': (0.025526315789473685, -0.07, -0.1, -0.06), 'RIVN': (0.005, None, -0.01, -0.16), 'RKT': (0.13842105263157894, None, -0.1, -0.06), 'SQ': (0.005, None, -0.01, -0.16), 'TOST': (0.005, None, -0.01, -0.16), 'TPR': (0.11789473684210527, -0.12000000000000001, None, -0.19), 'TSLA': (0.015263157894736843, -0.08000000000000002, -0.07, -0.01), 'TTD': (0.2, -0.11000000000000001, None, -0.03), 'W': (0.005, None, -0.01, -0.16)}
manual_params = {'ABNB': (0.004647348277747376, -0.7061235647895024, -0.7266265718972116), 'AFRM': (0.012978790756568603, -0.20861031972143085, -0.33523266856600187), 'AMAT': (0.037047632670576425, -0.8378868068468984, -0.8426548419396367), 'AMD': (0.053185887309110225, -0.07846234860452861, -0.21011058451816744), 'APA': (0.0039499670836076195, -0.14417379855167867, -0.17709019091507572), 'CAVA': (0.011761730112558544, -0.4308840268116858, -0.49411913494372073), 'CCL': (0.07415107415107425, -0.13374913374913378, -0.23769923769923773), 'COIN': (0.04519211626518027, -0.5769460481783795, -0.7013736810670914), 'CROX': (0.026034759544915076, -0.9267118028896489, -0.9302017170377609), 'CVNA': (0.008547008547008517, -0.5726495726495726, -0.6581196581196581), 'CZR': (0.008844665561083431, -0.19845218352681038, -0.2260917634051962), 'DASH': (0.0013932427725531493, -0.7910135841170324, -0.8258446534308603), 'DFS': (0.004862236628849326, -0.08427876823338742, -0.12479740680713136), 'GPS': (0.010544815465729274, -0.2970123022847101, -0.34094903339191573), 'KLAC': (0.016330906205187512, -0.9331727623317136, -0.9373494646859816), 'LRCX': (0.015927115422429194, -0.79244499792445, -0.8306788140962618), 'MGM': (0.010598964752279949, -0.7535124476213951, -0.7658368252403254), 'MRVL': (0.05155542573389815, -0.2697531765736819, -0.34277785891631374), 'MSTR': (0.003828950935212738, -0.9237598265112497, -0.9279953917050692), 'NVDA': (0.012483588864905748, -0.9877611873873473, -0.9899864260441933), 'ON': (0.050241271643485685, -0.2903775191598069, -0.3613397672438262), 'PATH': (0.0926118626430803, -0.16753381893860553, -0.24557752341311134), 'PINS': (0.03571428571428581, -0.40476190476190477, -0.4285714285714286), 'RBLX': (0.049284578696343395, -0.20508744038155802, -0.3322734499205088), 'RIVN': (0.0010010010010010895, -0.4494494494494494, -0.49949949949949946), 'RKT': (0.024140453547915053, -0.30504754937820044, -0.3416239941477688), 'SQ': (0.009676062263357155, -0.22871967465993548, -0.2988360678726687), 'TOST': (0.006336190831159216, -0.3663809168840849, -0.4036526276556094), 'TPR': (0.0731220364362366, -0.10157224856501124, -0.1265285749937609), 'TSLA': (0.05360004748619929, -0.7032112542292397, -0.7328901288063157), 'TTD': (0.042860417144166885, -0.7134998853999541, -0.7192298876919552), 'W': (0.0039592760180995334, -0.4343891402714932, -0.5050904977375565)}
all_stocks = list(set(all_stocks) & set(optimized_params.keys()))
print(sorted(all_stocks))

['ABNB', 'AFRM', 'AMAT', 'AMD', 'APA', 'CAVA', 'CCL', 'COIN', 'CROX', 'CVNA', 'CZR', 'DASH', 'DFS', 'GPS', 'KLAC', 'LRCX', 'MGM', 'MRVL', 'MSTR', 'NVDA', 'ON', 'PATH', 'PINS', 'RBLX', 'RIVN', 'RKT', 'SQ', 'TOST', 'TPR', 'TSLA', 'TTD', 'W']


In [ ]:
#@title get_pdf
def get_pdf(individual_csv_dir, individual_pdf_dir, final_pdf_path, manual_params, optimized_params, poscallbuy, poscallsell, posputsell, posputbuy, negcallbuy, negcallsell, negputsell, negputbuy, all_days):
    # Define paths
    os.makedirs(individual_csv_dir, exist_ok=True)
    os.makedirs(individual_pdf_dir, exist_ok=True)

    # Read the main data file
    df = pd.read_csv(main_data_file_path)

    # Function to round numerical values in the DataFrame
    def round_df(df):
        for col in df.columns:
            df[col] = df[col].apply(lambda x: round(float(x), 3) if is_number(x) else x)

        # Round specific cells to nearest dollar or 2 decimals as specified
        dollar_columns = ['Strike', '% change','5 day option price', 'Column 5', 'Column 6']
        dollar_rows = range(10, 17)
        for col in dollar_columns:
            for row in dollar_rows:
                if col in df.columns and is_number(df.at[row, col]):
                    df.at[row, col] = round(float(df.at[row, col]))

        specific_cells = [
            (0, 'Strike'), (1, 'Strike'), (2, 'Strike'), (3, 'Strike'), (4, 'Strike'), (6, 'Strike'),
            (1, '5 day option price'), (2, '5 day option price'), (3, '5 day option price'), (4, '5 day option price'), (5,'5 day option price')
        ]
        for row, col in specific_cells:
            if col in df.columns and is_number(df.at[row, col]):
                df.at[row, col] = round(float(df.at[row, col]), 2)

        percent_cells = [
            (1, '% change'), (2, '% change'), (3, '% change'), (4, '% change'), (5, '% change'), (6, '% change')
        ]
        for row, col in percent_cells:
            if col in df.columns and is_number(df.at[row, col]):
                df.at[row, col] = f"{round(float(df.at[row, col]) * 100, 2)}%"

        percent_columns = ['Strike', '% change','5 day option price', 'Column 6']
        percent_rows = range(19,26)
        for col in percent_columns:
            for row in percent_rows:
                if col in df.columns and is_number(df.at[row, col]):
                    df.at[row, col] = f"{round(float(df.at[row, col]) * 100, 2)}%"

        percent_columns = ['Strike', '% change','5 day option price']
        percent_rows = range(28,35)
        for col in percent_columns:
            for row in percent_rows:
                if col in df.columns and is_number(df.at[row, col]):
                    df.at[row, col] = f"{round(float(df.at[row, col]) * 100, 2)}%"

        return df


    # Helper function to check if a value is a number
    def is_number(value):
        try:
            float(value)
            return True
        except (ValueError, TypeError):
            return False

    def add_label(df, label):
        # Create a label row
        label_row = pd.DataFrame([[label] + [''] * (df.shape[1] - 1)], columns=df.columns)
        # Concatenate the label row with the DataFrame
        labeled_df = pd.concat([label_row, df], ignore_index=True)
        return labeled_df


    def csv_to_pdf(csv_path, pdf_path, fontsize=10):
        df = pd.read_csv(csv_path)

        # Replace NaN values with empty strings
        df = df.fillna('')

        fig, ax = plt.subplots(figsize=(12, 8))  # Adjust size as needed
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

        # Set font size for the table
        table.auto_set_font_size(False)
        table.set_fontsize(fontsize)

        # Highlight the top left cell if ante is positive
        if not df.iloc[6, 2].startswith('-'):
            cell = table[0, 0]
            cell.set_facecolor('green')
            cell.set_text_props(color='white')

        # Set specific width for the first column and enable text wrapping
        cell_dict = table.get_celld()
        for i in range(len(df.index) + 1):
            cell_dict[(i, 0)].set_width(0.25)
            cell_dict[(i, 0)].get_text().set_wrap(True)

        # Set borders and linewidth
        for key, cell in table.get_celld().items():
            cell.set_edgecolor('black')
            cell.set_linewidth(0.5)

        # Extract relevant values, check for None
        def parse_float(value):
            try:
                return float(value)
            except ValueError:
                return None

        pp = PdfPages(pdf_path)
        pp.savefig(fig, bbox_inches='tight')
        pp.close()
        plt.close(fig)



    def merge_pdfs(pdf_list, output_path):
        merger = PyPDF2.PdfMerger()
        for pdf in pdf_list:
            merger.append(pdf)
        merger.write(output_path)
        merger.close()

    pdf_files = []

    # Assuming all_stocks is already defined
    for stock in all_stocks:
        manual_df = get_df(stock, manual_params[stock][0], None, manual_params[stock][1], manual_params[stock][2], all_days)
        individual_df = get_df(stock, optimized_params[stock][0], optimized_params[stock][1], optimized_params[stock][2], optimized_params[stock][3], all_days)
        pos_portfolio_df = get_df(stock, poscallbuy, poscallsell, posputsell, posputbuy, all_days)
        neg_portfolio_df = get_df(stock, negcallbuy, negcallsell, negputsell, negputbuy, all_days)
        if pos_portfolio_df.iloc[time_frame,5] > neg_portfolio_df.iloc[time_frame,5]:
            portfolio_df = pos_portfolio_df
        else:
            portfolio_df = neg_portfolio_df

        manual_df = round_df(manual_df)
        individual_df = round_df(individual_df)
        portfolio_df = round_df(portfolio_df)

        # Add labels to each DataFrame
        manual_df = add_label(manual_df, 'Manual Selection')
        individual_df = add_label(individual_df, 'Learned Stock')
        portfolio_df = add_label(portfolio_df, 'Learned Portfolio')

        # Save to CSV
        csv_manual_path = os.path.join(individual_csv_dir, f'{stock}_manual_{today_date_str}_{expr_date_str}.csv')
        manual_df.to_csv(csv_manual_path, index=False)
        csv_individual_path = os.path.join(individual_csv_dir, f'{stock}_individual_{today_date_str}_{expr_date_str}.csv')
        individual_df.to_csv(csv_individual_path, index=False)
        csv_portfolio_path = os.path.join(individual_csv_dir, f'{stock}_portfolio_{today_date_str}_{expr_date_str}.csv')
        portfolio_df.to_csv(csv_portfolio_path, index=False)

        # Convert CSV to PDF
        pdf_manual_path = os.path.join(individual_pdf_dir, f'{stock}_manual_{today_date_str}_{expr_date_str}.pdf')
        csv_to_pdf(csv_manual_path, pdf_manual_path, fontsize=10)
        pdf_individual_path = os.path.join(individual_csv_dir, f'{stock}_individual_{today_date_str}_{expr_date_str}.pdf')
        csv_to_pdf(csv_individual_path, pdf_individual_path, fontsize=10)
        pdf_portfolio_path = os.path.join(individual_csv_dir, f'{stock}_portfolio_{today_date_str}_{expr_date_str}.pdf')
        csv_to_pdf(csv_portfolio_path, pdf_portfolio_path, fontsize=10)

        pdf_files.append(pdf_manual_path)
        pdf_files.append(pdf_individual_path)
        pdf_files.append(pdf_portfolio_path)

    # Merge all PDFs into one
    merge_pdfs(pdf_files, final_pdf_path)
    print(f"All PDFs merged into {final_pdf_path}")


In [ ]:
#@title run get_pdf
get_pdf('individual_csvs', 'individual_pdfs', f'SignalDays_{today_date_str}_{expr_date_str}.pdf', manual_params, optimized_params, poscallbuy, poscallsell, posputsell, posputbuy, negcallbuy, negcallsell, negputsell, negputbuy, all_days=False)

All PDFs merged into SignalDays_05-24-2024_05-31-2024.pdf


In [ ]:
#@title get recommendations

# Function to get user input for threshold
def get_return_threshold():
    while True:
        threshold_input = input("Enter the return threshold percentage (e.g., 10 for 10%) or press Enter to include all trades: ")
        if threshold_input == "":
            return None
        try:
            return float(threshold_input) / 100
        except ValueError:
            print("Invalid input. Please enter a number or press Enter to include all trades.")

# Get the return threshold
return_threshold = get_return_threshold()

# Initialize the final list of trades
final_trades = []

# Assuming all_stocks is already defined
for stock in all_stocks:

    # Generate DataFrames for the stock
    manual_df = get_df(stock, manual_params[stock][0], None, manual_params[stock][1], manual_params[stock][2])
    individual_df = get_df(stock, optimized_params[stock][0], optimized_params[stock][1], optimized_params[stock][2], optimized_params[stock][3])
    pos_portfolio_df = get_df(stock, poscallbuy, poscallsell, posputsell, posputbuy)
    neg_portfolio_df = get_df(stock, negcallbuy, negcallsell, negputsell, negputbuy)

    if pos_portfolio_df.iloc[time_frame, 5] > neg_portfolio_df.iloc[time_frame, 5]:
        portfolio_df = pos_portfolio_df
    else:
        portfolio_df = neg_portfolio_df

    # Find the trades for the current stock
    trades = [
        ('Manual Selection', manual_df),
        ('Learned Stock', individual_df),
        ('Learned Portfolio', portfolio_df)
    ]

    for strategy_name, trade_df in trades:
        trade_return = trade_df.iloc[time_frame, 5]

        if return_threshold is None or trade_return > return_threshold:
            call_buy_strike = trade_df.iloc[1, 1]
            call_sell_strike = trade_df.iloc[2, 1]
            put_sell_strike = trade_df.iloc[3, 1]
            put_buy_strike = trade_df.iloc[4, 1]
            ante = trade_df.iloc[5, 3]
            projected_return = trade_return

            final_trades.append({
                'Stock': stock,
                'Strategy': strategy_name,
                'Call Buy Strike': call_buy_strike,
                'Call Sell Strike': call_sell_strike,
                'Put Sell Strike': put_sell_strike,
                'Put Buy Strike': put_buy_strike,
                'Ante': ante,
                'Projected Return': projected_return
            })

# Define the CSV file path
csv_file_path = 'final_trades.csv'

# Write final trades to a CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Stock', 'Strategy', 'Call Buy Strike', 'Call Sell Strike', 'Put Sell Strike', 'Put Buy Strike', 'Ante', 'Projected Return'])
    writer.writeheader()
    for trade in final_trades:
        writer.writerow(trade)

print(f"Final trades saved to {csv_file_path}")

Enter the return threshold percentage (e.g., 10 for 10%) or press Enter to include all trades: 
Final trades saved to final_trades.csv




---



In [ ]:
#@title run individual optimize (all days)
all_optimized_params, all_validation_gains, all_average_validation_gain = individual_optimize(all_days=True)

Average gain after ante for validation sets across all stocks: 16.78250738350836


In [ ]:
#@title run portfolio optimize (all days)
all_train_gain_post_ante, all_train_gain_neg_ante, all_val_gain_pos_ante, all_val_gain_neg_ante, all_poscallbuy, all_poscallsell, all_posputsell, all_posputbuy, all_negcallbuy, all_negcallsell, all_negputsell, all_negputbuy = portfolio_optimize(all_days=True)

In [ ]:
#@title Run manual optimization (all days)
all_manual_params, all_manual_validation_gains, all_average_manual_gain = manual_optimize(all_days=True)

Average %gain after ante for validation sets across all stocks: -2.779186324253009


In [ ]:
all_poscallbuy = 0.0666
all_poscallsell = -0.01
all_posputsell = -0.1
all_posputbuy = -0.03
all_negcallbuy = 0.128
all_negcallsell = -0.13
all_negputsell = -0.08
all_negputbuy = -0.2
all_manual_params = {'ABNB': (0.004647348277747376, -0.7061235647895024, -0.7266265718972116), 'AFRM': (0.012978790756568603, -0.20861031972143085, -0.33523266856600187), 'AMAT': (0.037047632670576425, -0.8378868068468984, -0.8426548419396367), 'AMD': (0.053185887309110225, -0.07846234860452861, -0.21011058451816744), 'APA': (0.0039499670836076195, -0.14417379855167867, -0.17709019091507572), 'CAVA': (0.04970279499177943, -0.4308840268116858, -0.49411913494372073), 'CCL': (0.10880110880110871, -0.13374913374913378, -0.23769923769923773), 'COIN': (0.04519211626518027, -0.5769460481783795, -0.7013736810670914), 'CROX': (0.026034759544915076, -0.9267118028896489, -0.9302017170377609), 'CVNA': (0.008547008547008517, -0.5726495726495726, -0.6581196581196581), 'CZR': (0.008844665561083431, -0.19845218352681038, -0.2260917634051962), 'DASH': (0.0013932427725531493, -0.7910135841170324, -0.8258446534308603), 'DFS': (0.004862236628849326, -0.08427876823338742, -0.10048622366288495), 'GPS': (0.010544815465729274, -0.2970123022847101, -0.34094903339191573), 'KLAC': (0.016330906205187512, -0.9331727623317136, -0.9373494646859816), 'LRCX': (0.015927115422429194, -0.79244499792445, -0.8306788140962618), 'MGM': (0.010598964752279949, -0.7535124476213951, -0.7658368252403254), 'MRVL': (0.05155542573389815, -0.2697531765736819, -0.34277785891631374), 'MSTR': (0.016535646516671276, -0.9237598265112497, -0.9279953917050692), 'NVDA': (0.012483588864905748, -0.9877611873873473, -0.9899864260441933), 'ON': (0.050241271643485685, -0.2903775191598069, -0.3613397672438262), 'PATH': (0.0926118626430803, -0.16753381893860553, -0.24557752341311134), 'PINS': (0.011904761904761862, -0.40476190476190477, -0.4285714285714286), 'RBLX': (0.049284578696343395, -0.20508744038155802, -0.3322734499205088), 'RIVN': (0.10110110110110115, -0.4494494494494494, -0.49949949949949946), 'RKT': (0.09729334308705195, -0.30504754937820044, -0.3416239941477688), 'SQ': (0.009676062263357155, -0.22871967465993548, -0.2988360678726687), 'TOST': (0.006336190831159216, -0.3663809168840849, -0.4036526276556094), 'TPR': (0.09807836286498617, -0.10157224856501124, -0.1265285749937609), 'TSLA': (0.05360004748619929, -0.7032112542292397, -0.7328901288063157), 'TTD': (0.042860417144166885, -0.7134998853999541, -0.7192298876919552), 'W': (0.0039592760180995334, -0.4343891402714932, -0.5050904977375565)}
all_optimized_params = {'ABNB': (0.005, None, -0.01, -0.16), 'AFRM': (0.005, None, -0.049999999999999996, -0.13), 'AMAT': (0.2, -0.11000000000000001, None, -0.03), 'AMD': (0.11789473684210527, -0.12000000000000001, None, -0.19), 'APA': (0.18973684210526315, None, -0.1, -0.01), 'CAVA': (0.15894736842105264, -0.17, -0.03, -0.09000000000000001), 'CCL': (0.13842105263157894, -0.19, None, -0.06), 'COIN': (0.18973684210526315, -0.2, None, -0.01), 'CROX': (0.13842105263157894, -0.19, None, -0.06), 'CVNA': (0.005, None, -0.01, -0.16), 'CZR': (0.005, None, -0.01, -0.16), 'DASH': (0.005, None, -0.01, -0.16), 'DFS': (0.005, -0.020000000000000018, -0.049999999999999996, -0.13), 'GPS': (0.005, None, -0.01, -0.16), 'KLAC': (0.2, -0.11000000000000001, None, -0.03), 'LRCX': (0.18973684210526315, -0.2, None, -0.01), 'MGM': (0.005, None, -0.01, -0.16), 'MRVL': (0.2, -0.11000000000000001, None, -0.03), 'MSTR': (0.2, -0.11000000000000001, None, -0.03), 'NVDA': (0.18973684210526315, -0.2, None, -0.01), 'ON': (0.11789473684210527, -0.12000000000000001, None, -0.19), 'PATH': (0.005, -0.020000000000000018, -0.049999999999999996, -0.13), 'PINS': (0.005, None, -0.01, -0.16), 'RBLX': (0.025526315789473685, -0.07, -0.1, -0.06), 'RIVN': (0.09736842105263158, -0.03, -0.039999999999999994, -0.11000000000000001), 'RKT': (0.025526315789473685, -0.07, -0.1, -0.06), 'SQ': (0.005, None, -0.01, -0.16), 'TOST': (0.005, None, -0.01, -0.16), 'TPR': (0.11789473684210527, -0.12000000000000001, None, -0.19), 'TSLA': (0.1281578947368421, -0.08000000000000002, None, -0.2), 'TTD': (0.2, -0.11000000000000001, None, -0.03), 'W': (0.005, None, -0.01, -0.16)}

In [ ]:
#@title run get_pdf
get_pdf('alldays_csvs', 'alldays_pdfs', f'AllDays_{today_date_str}_{expr_date_str}.pdf', all_manual_params, all_optimized_params, all_poscallbuy, all_poscallsell, all_posputsell, all_posputbuy, all_negcallbuy, all_negcallsell, all_negputsell, all_negputbuy, all_days=True)

All PDFs merged into AllDays_05-31-2024_2024-06-14.pdf


In [ ]:
#@title Are the signal days better than the average day?
# Define paths
individual_csv_dir_signal = 'individual_csvs'
individual_csv_dir_all_days = 'alldays_csvs'

results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)

# Helper function to check if a value is a number
def is_number(value):
    try:
        float(value.strip('%')) / 100.0
        return True
    except (ValueError, TypeError):
        return False

# Extract average returns from CSV files
def extract_average_returns(csv_path):
    df = pd.read_csv(csv_path)
    averages = {}
    time_frames = {
        '1 Mo': 20,
        'YTD': 21,
        'TTM': 22,
        '2023': 23,
        '2022': 24,
        '2021': 25,
        'All': 26
    }
    for tf, row in time_frames.items():
        value = df.iloc[row, 5]
        num = float(df.iloc[row, 4])
        if is_number(value) and num != 0:
            averages[tf] = (float(value.strip('%')) / 100.0) / num
        else:
            averages[tf] = 0
    return averages

# Initialize a list to hold the results for each strategy
results_manual = []
results_individual = []
results_portfolio = []

# Assuming all_stocks is already defined
for stock in all_stocks:
    for strategy in ['manual', 'individual', 'portfolio']:
        # Paths to CSV files
        csv_signal_path = os.path.join(individual_csv_dir_signal, f'{stock}_{strategy}_{today_date_str}_{expr_date_str}.csv')
        csv_all_days_path = os.path.join(individual_csv_dir_all_days, f'{stock}_{strategy}_{today_date_str}_{expr_date_str}.csv')

        if os.path.exists(csv_signal_path) and os.path.exists(csv_all_days_path):
            # Extract average returns for signal days and all days
            signal_averages = extract_average_returns(csv_signal_path)
            all_days_averages = extract_average_returns(csv_all_days_path)

            # Append the results to the appropriate list
            result = {
                'Stock': stock,
                **{f'Signal {tf}': signal_averages[tf] for tf in signal_averages},
                **{f'All Days {tf}': all_days_averages[tf] for tf in all_days_averages}
            }

            if strategy == 'manual':
                results_manual.append(result)
            elif strategy == 'individual':
                results_individual.append(result)
            elif strategy == 'portfolio':
                results_portfolio.append(result)

# Convert the results to DataFrames
results_df_manual = pd.DataFrame(results_manual)
results_df_individual = pd.DataFrame(results_individual)
results_df_portfolio = pd.DataFrame(results_portfolio)

# Restructure the columns
time_frames = ['1 Mo', 'YTD', 'TTM', '2023', '2022', '2021', 'All']
ordered_columns = ['Stock']
for tf in time_frames:
    ordered_columns.append(f'Signal {tf}')
    ordered_columns.append(f'All Days {tf}')

results_df_manual = results_df_manual[ordered_columns]
results_df_individual = results_df_individual[ordered_columns]
results_df_portfolio = results_df_portfolio[ordered_columns]


# Calculate the sum of returns for each time period
def add_sum_row(df):
    sum_row = {col: df[col].sum() if df[col].dtype in [float, int] else '' for col in df.columns}
    sum_row['Stock'] = 'Total'
    return pd.concat([pd.DataFrame([sum_row]), df], ignore_index=True)

results_df_manual = add_sum_row(results_df_manual)
results_df_individual = add_sum_row(results_df_individual)
results_df_portfolio = add_sum_row(results_df_portfolio)

# Round all values to 3 decimal points
results_df_manual = results_df_manual.round(3)
results_df_individual = results_df_individual.round(3)
results_df_portfolio = results_df_portfolio.round(3)


results_csv_path_manual = os.path.join(results_dir, 'average_returns_manual.csv')
results_csv_path_individual = os.path.join(results_dir, 'average_returns_individual.csv')
results_csv_path_portfolio = os.path.join(results_dir, 'average_returns_portfolio.csv')

# Save the results to CSV files
if not results_df_manual.empty:
    results_df_manual.to_csv(results_csv_path_manual, index=False)
    print(f"Average returns saved to {results_csv_path_manual}")

if not results_df_individual.empty:
    results_df_individual.to_csv(results_csv_path_individual, index=False)
    print(f"Average returns saved to {results_csv_path_individual}")

if not results_df_portfolio.empty:
    results_df_portfolio.to_csv(results_csv_path_portfolio, index=False)
    print(f"Average returns saved to {results_csv_path_portfolio}")

# Function to create highlighted PDF
def create_highlighted_pdf(results_df, pdf_path, strategy_name):
    if results_df.empty:
        print(f"No data to create PDF for {strategy_name}")
        return

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=results_df.values, colLabels=results_df.columns, cellLoc='center', loc='center')

    # Highlight the better returns
    for i in range(1, len(results_df.columns), 2):
        for j in range(len(results_df)):
            signal_value = results_df.iloc[j, i]
            all_days_value = results_df.iloc[j, i + 1]
            if signal_value > all_days_value:
                table[(j + 1, i)].set_facecolor('lightgreen')
            elif all_days_value > signal_value:
                table[(j + 1, i + 1)].set_facecolor('lightcoral')

    plt.title(f'{strategy_name} Average Returns Comparison')
    pp = PdfPages(pdf_path)
    pp.savefig(fig, bbox_inches='tight')
    pp.close()
    plt.close(fig)

# Define the PDF file paths
results_pdf_path_manual = os.path.join(results_dir, 'average_returns_manual.pdf')
results_pdf_path_individual = os.path.join(results_dir, 'average_returns_individual.pdf')
results_pdf_path_portfolio = os.path.join(results_dir, 'average_returns_portfolio.pdf')

# Create PDFs with highlighted better returns
create_highlighted_pdf(results_df_manual, results_pdf_path_manual, 'Manual')
create_highlighted_pdf(results_df_individual, results_pdf_path_individual, 'Individual')
create_highlighted_pdf(results_df_portfolio, results_pdf_path_portfolio, 'Portfolio')


# Function to merge PDFs
def merge_pdfs(pdf_list, output_path):
    merger = PyPDF2.PdfMerger()
    for pdf in pdf_list:
        merger.append(pdf)
    merger.write(output_path)
    merger.close()

# Merge all PDFs into one
final_pdf_path = 'average_returns_combined.pdf'
pdf_files = [results_pdf_path_manual, results_pdf_path_individual, results_pdf_path_portfolio]
merge_pdfs(pdf_files, final_pdf_path)
print(f"All PDFs merged into {final_pdf_path}")


Average returns saved to results/average_returns_manual.csv
Average returns saved to results/average_returns_individual.csv
Average returns saved to results/average_returns_portfolio.csv
All PDFs merged into average_returns_combined.pdf


In [ ]:
#@title Graphing
# Function to upload multiple CSV files
def upload_files():
    uploaded = files.upload()
    file_paths = list(uploaded.keys())
    return file_paths

# Function to compute the center of a cluster
def compute_center(data):
    return data.mean()

# Function to compute variance
def compute_variance(data):
    return data.var()

# Get user inputs for percentage and ante thresholds
def get_user_inputs():
    percentage_threshold = float(input("Enter the percentage threshold: "))
    ante_threshold = float(input("Enter the ante threshold: "))
    variance_threshold = float(input("Enter the variance threshold: "))
    return percentage_threshold, ante_threshold, variance_threshold

# Upload the CSV files
print("Please upload the CSV files.")
csv_file_paths = upload_files()

# Initialize a dictionary to hold the aggregated data for each stock
stock_data = {}

# Process each uploaded CSV file
for file_path in csv_file_paths:
    df = pd.read_csv(file_path)

    # Ensure 'Ante' and 'Projected Return' columns are numeric
    df['Ante'] = pd.to_numeric(df['Ante'], errors='coerce')
    df['Projected Return'] = pd.to_numeric(df['Projected Return'], errors='coerce')

    # Group data by stock
    for stock, group in df.groupby('Stock'):
        if stock not in stock_data:
            stock_data[stock] = []
        stock_data[stock].append(group)

# Combine data for each stock
for stock in stock_data:
    stock_data[stock] = pd.concat(stock_data[stock])

# Get user inputs
percentage_threshold, ante_threshold, variance_threshold = get_user_inputs()

# Initialize a dictionary to hold the filtered data
filtered_stock_data = {}

# Define colors for each strategy
strategy_colors = {
    'Manual Selection': 'red',
    'Learned Stock': 'green',
    'Learned Portfolio': 'blue'
}

# Define common axis limits
min_ante = float('inf')
max_ante = float('-inf')
min_return = float('inf')
max_return = float('-inf')

# Calculate the common axis limits
for stock, data in stock_data.items():
    min_ante = min(min_ante, data['Ante'].min())
    max_ante = max(max_ante, data['Ante'].max())
    min_return = min(min_return, data['Projected Return'].min())
    max_return = max(max_return, data['Projected Return'].max())

# Create a directory to save the plots
output_dir = 'stock_plots'
os.makedirs(output_dir, exist_ok=True)

# Create a PDF file to save the plots
pdf_path = os.path.join(output_dir, 'stock_ante_vs_return.pdf')
pdf = PdfPages(pdf_path)

# Initialize a dictionary to hold the centers for each strategy and stock
centers = {}

# Plot the graphs for each stock
for stock, data in stock_data.items():
    plt.figure(figsize=(10, 6))

    # Initialize the filtered data for this stock
    filtered_stock_data[stock] = []

    # Plot each strategy
    for strategy, group in data.groupby('Strategy'):
        ante_mean = group['Ante'].mean()
        return_mean = group['Projected Return'].mean() * 100
        variance = compute_variance(group['Projected Return'])

        # Check the thresholds
        if return_mean >= percentage_threshold and ante_mean >= ante_threshold and variance <= variance_threshold:
            plt.scatter(group['Ante'], group['Projected Return'] * 100,
                        color=strategy_colors.get(strategy, 'black'), label=strategy)

            # Store the filtered data
            filtered_stock_data[stock].append(group)

            # Save the center of the cluster
            if stock not in centers:
                centers[stock] = []
            centers[stock].append({'Strategy': strategy, 'Ante': ante_mean, 'Return': return_mean, 'Variance': variance})

    plt.xlabel('Ante')
    plt.ylabel('Projected Return (%)')
    plt.title(f'{stock} - Ante vs. Projected Return')
    plt.legend()
    plt.grid(True)
    plt.xlim(min_ante, max_ante)
    plt.ylim(min_return * 100, max_return * 100)

    # Save the plot to the PDF
    pdf.savefig()
    plt.close()

# Close the PDF file
pdf.close()

print(f"Plots saved to {pdf_path}")

# Filter out the discarded strategies from the CSV files and save to new CSVs
filtered_csv_dir = 'filtered_csvs'
os.makedirs(filtered_csv_dir, exist_ok=True)

for file_path in csv_file_paths:
    df = pd.read_csv(file_path)

    filtered_df = pd.DataFrame()

    # Group data by stock and strategy
    for stock, group in df.groupby('Stock'):
        if stock in centers:
            for strategy in group['Strategy'].unique():
                if any(center['Strategy'] == strategy for center in centers[stock]):
                    filtered_df = pd.concat([filtered_df, group[group['Strategy'] == strategy]])

    # Save the filtered data to a new CSV file
    new_file_path = os.path.join(filtered_csv_dir, os.path.basename(file_path))
    filtered_df.to_csv(new_file_path, index=False)

print(f"Filtered CSV files saved to {filtered_csv_dir}.")


Please upload the CSV files.


Saving final_trades (2).csv to final_trades (2) (2).csv
Enter the percentage threshold: 100
Enter the ante threshold: 0
Enter the variance threshold: 100


Plots saved to stock_plots/stock_ante_vs_return.pdf
Filtered CSV files saved to filtered_csvs.


In [ ]:
#@title Actual returns
# Function to calculate the actual return based on June 14th prices
def calculate_actual_returns(trades_df, stock_prices):
    actual_returns = []

    for _, row in trades_df.iterrows():
        stock = row['Stock']
        strategy = row['Strategy']
        call_buy_strike = row['Call Buy Strike']
        call_sell_strike = row['Call Sell Strike']
        put_sell_strike = row['Put Sell Strike']
        put_buy_strike = row['Put Buy Strike']
        ante = row['Ante']

        stock_price = stock_prices.get(stock, None)

        if stock_price is None:
            actual_returns.append(None)
            continue

        # Calculate the return for each trade
        if stock_price > call_buy_strike:
            call_buy_return = stock_price - call_buy_strike
        else:
            call_buy_return = 0

        if stock_price > call_sell_strike:
            call_sell_return = stock_price - call_sell_strike
        else:
            call_sell_return = 0

        if stock_price < put_sell_strike:
            put_sell_return = put_sell_strike - stock_price
        else:
            put_sell_return = 0

        if stock_price < put_buy_strike:
            put_buy_return = put_buy_strike - stock_price
        else:
            put_buy_return = 0

        total_return_dollars = call_buy_return - call_sell_return - put_sell_return + put_buy_return + ante
        actual_returns.append(total_return_dollars)

    trades_df['Actual Return'] = actual_returns
    return trades_df


# Define the dictionary of stock prices for June 14th
# june_14th_prices = {
#     'ABNB':145.97,
#     'AFRM':30.90,
#     'AMAT':237.03,
#     'AMD':159.63,
#     'APA':27.82,
#     'CAVA':89.93,
#     'CCL':15.34,
#     'COIN':244.5,
#     'CROX':156.98,
#     'CVNA':103.06,
#     'CZR':36.03,
#     'DASH':112.05,
#     'DFS':122,
#     'GPS':24.99,
#     'LRCX':1035.98,
#     'MGM':39.85,
#     'MRVL':73.27,
#     'ON':71.97,
#     'PATH':11.54,
#     'PINS':43.51,
#     'RBLX':35.1,
#     'RIVN':10.88,
#     'RKT':14.69,
#     'SQ':62.15,
#     'TOST':22.72,
#     'TPR':41.81,
#     'TSLA':178.01,
#     'TTD':95.66,
#     'W':52.62
# }

may_31_prices = {
    'ABNB': 144.93,
    'AFRM': 29.27,
    'AMAT': 215.08,
    'AMD': 166.9,
    'APA': 30.53,
    'CAVA': 92.55,
    'CCL': 15.08,
    'COIN': 225.92,
    'CROX': 155.64,
    'CVNA': 99.98,
    'CZR': 35.56,
    'DASH': 110.11,
    'DFS': 122.66,
    'GPS': 28.96,
    'KLAC': 759.53,
    'LRCX': 932.44,
    'MGM': 40.17,
    'MRVL': 68.81,
    'MSTR': 1524.49,
    'NVDA': 109.63,
    'ON': 73.04,
    'PATH': 12.26,
    'PINS': 41.49,
    'RBLX': 33.62,
    'RIVN': 10.92,
    'RKT': 13.9,
    'SQ': 64.08,
    'TOST': 24.23,
    'TPR': 43.49,
    'TSLA': 178.08,
    'TTD': 92.78,
    'W': 59.49


}
# Define the path for the final trades CSV
final_trades_file_path = 'final_trades.csv'
updated_csv_dir = 'updated_csvs'
os.makedirs(updated_csv_dir, exist_ok=True)

# Process the final trades CSV file
trades_df = pd.read_csv(final_trades_file_path)
trades_df = calculate_actual_returns(trades_df, may_31_prices)

# Print the sum of actual returns for each strategy
strategy_sums = trades_df.groupby('Strategy')['Actual Return'].sum()
print("Sum of Actual Returns for each Strategy:")
print(strategy_sums)

# Save the updated DataFrame to a new CSV file
new_file_path = os.path.join(updated_csv_dir, os.path.basename(final_trades_file_path))
trades_df.to_csv(new_file_path, index=False)

print(f"Updated CSV file with actual returns saved to {new_file_path}.")

# filtered_csv_dir = 'individual_csvs'
# updated_csv_dir = 'updated_csvs'
# os.makedirs(updated_csv_dir, exist_ok=True)

# # Process each final trades CSV file
# for file_name in os.listdir(filtered_csv_dir):
#     file_path = os.path.join(filtered_csv_dir, file_name)
#     trades_df = pd.read_csv(file_path)
#     trades_df = calculate_actual_returns(trades_df, may_31_prices)

#     # Save the updated DataFrame to a new CSV file
#     new_file_path = os.path.join(updated_csv_dir, os.path.basename(file_path))
#     trades_df.to_csv(new_file_path, index=False)

# print(f"Updated CSV files with actual returns saved to {updated_csv_dir}.")


Sum of Actual Returns for each Strategy:
Strategy
Learned Portfolio     215.265
Learned Stock        1088.595
Manual Selection     -221.800
Name: Actual Return, dtype: float64
Updated CSV file with actual returns saved to updated_csvs/final_trades.csv.


In [ ]:
#@title ignore (graphing maxloss)
# import matplotlib.pyplot as plt
# import numpy as np

# # Define the parameters for your options
# current_price = 100
# call_buy_strike = 105
# call_sell_strike = 110
# put_buy_strike = 95
# put_sell_strike = 90

# # Define the range of stock prices to plot
# prices = np.linspace(70, 130, 500)

# # Calculate the payoff for each price point
# def calculate_payoff(price, call_buy_strike, call_sell_strike, put_buy_strike, put_sell_strike):
#     payoff = 0

#     # Call buy
#     if call_buy_strike is not None:
#         payoff += max(price - call_buy_strike, 0)

#     # Call sell
#     if call_sell_strike is not None:
#         payoff -= max(price - call_sell_strike, 0)

#     # Put buy
#     if put_buy_strike is not None:
#         payoff += max(put_buy_strike - price, 0)

#     # Put sell
#     if put_sell_strike is not None:
#         payoff -= max(put_sell_strike - price, 0)

#     return payoff

# payoffs = [calculate_payoff(price, call_buy_strike, call_sell_strike, put_buy_strike, put_sell_strike) for price in prices]

# # Create the plot
# plt.figure(figsize=(10, 6))
# plt.plot(prices, payoffs, label='Payoff')
# plt.axhline(0, color='black', linestyle='--', linewidth=1)
# plt.axvline(current_price, color='red', linestyle='--', linewidth=1, label='Current Price')
# plt.axvline(call_buy_strike, color='green', linestyle='--', linewidth=1, label='Call Buy Strike')
# plt.axvline(call_sell_strike, color='blue', linestyle='--', linewidth=1, label='Call Sell Strike')
# plt.axvline(put_buy_strike, color='purple', linestyle='--', linewidth=1, label='Put Buy Strike')
# plt.axvline(put_sell_strike, color='orange', linestyle='--', linewidth=1, label='Put Sell Strike')

# # Add labels and title
# plt.xlabel('Stock Price')
# plt.ylabel('Payoff')
# plt.title('Options Strategy Payoff Diagram')
# plt.legend()
# plt.grid(True)
# plt.show()
